In [1]:
%matplotlib inline

from __future__ import division, print_function

import glob as glob

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import optimize

sns.set_style('white')

### Set parameters

In [2]:
N_samples = 10000
threshold = .5 
REWARD = 1

_results_index = [
    'High Effort Success', 'Low Effort Success', 'High Effort Failure', 
    'Low Effort Failure', 'Baseline'
]

human_means = pd.Series([107, 57, 30, 35,85], index=_results_index)

### Free parameters to fit

In [3]:
#COST = 0.5
#B=6
#A=60 
#u_skill = .625 #mean of beta
#tao_skill = 20 #variance of beta
#skill=.4
#scale = 200

### Functions

In [4]:
def prob_reward(e,diff,skill, A, B):
    """Return the probability of getting the reward
    
    Parameters
    ----------
    e : numeric or array-like
        Description of this parameter.
    # TODO: list other parameters
    
    Returns
    -------
    ...
    """
    return 1/(1 + A * np.exp(-(e * skill / diff) * B)) 


def get_effort(cost, diff, skill, A, B,reward=1):
    """Documentation."""
    es = np.linspace(0, 1, 100) 
    utilities = (
        reward * prob_reward(e=es, diff=diff, skill=skill, A=A, B=B)
        - cost * es
    )
    return es[np.argmax(utilities)]  #return argmax of utility


def get_effort_multidim(cost, diff, skill, A, B, reward=1):
    """Similar to `get_effort` but accepts arrays or single values of `diff` 
    and `skill`.

    Parameters
    ----------
    ...

    Returns
    -------
    Ndarray with same shape as `diff` and `skill`.
    """
    es = np.linspace(0, 1, 100)
    es_rank2 = es[np.newaxis, ...]
    
    try:
        diff = diff[..., np.newaxis]
    except TypeError:
        pass
    try:
        skill = skill[..., np.newaxis]
    except TypeError:
        pass
    
    prob = prob_reward(e=es_rank2, diff=diff, skill=skill, A=A, B=B)
    utilities = reward * prob - cost * es_rank2
    return es[utilities.argmax(-1)]


def your_eff_know_skill(n_samples, cost, diff, skill, A, B,reward=1):  #model for planning. Takes in diff, skill, and cost -returns effort
    """Documentation."""
    sampled_effort = np.empty(n_samples)
    for i in range(n_samples):
        d = np.random.choice(diff)
        effort=get_effort(cost=cost, diff=d, skill=skill, A=A, B=B, reward=reward)
        sampled_effort[i] = effort
    
    return sampled_effort


def your_eff_know_skill_opt(n_samples, cost, diff, skill, A, B,reward=1):
    """Model for planning.

    Parameters
    ----------
    n_samples : int
    cost : numeric
    diff : array-like
    skill : numeric
    A : numeric
    B : numeric
    reward : numeric
    
    Returns
    -------
    Ndarray of effort.
    """
    rand_diffs = np.random.choice(diff, size=n_samples)
    return get_effort_multidim(
        cost=cost, diff=rand_diffs, skill=skill, A=A, B=B, reward=reward
    )


def create_priors(n_samples, tao_skill, u_skill, cost, A, B, reward=1,threshold=0.5):
    """Documentation."""
    # Skill and diff are beta distributions.
    skills = np.random.beta(
        tao_skill * u_skill, (1 - u_skill) * tao_skill, size=n_samples,
    )
    diffs = np.random.beta(.5,.5, size=n_samples)

    # Create array of effort values, for each value of skill and diff.
    # This initializes an empty array and fills in a value with each 
    # iteration.
    # Please see `create_priors_opt` for a vectorized approach (faster).
    efforts = np.empty_like(skills)
    for ii, (this_skill, this_diff) in enumerate(zip(skills, diffs)):
        efforts[ii] = get_effort(
            cost=cost, skill=this_skill, diff=this_diff, A=A, B=B
        )

    high_effort = efforts > threshold
    success = prob_reward(efforts, diffs, skills, A, B) > np.random.random(n_samples)

    return skills, diffs, success, high_effort


def create_priors_opt(n_samples, tao_skill, u_skill, cost, A, B, reward=1,threshold=0.5):
    """Documentation."""
    # Skill and diff are beta distributions.
    skills = np.random.beta(
        tao_skill * u_skill, (1 - u_skill) * tao_skill, size=n_samples,
    )
    diffs = np.random.beta(.5,.5, size=n_samples)

    efforts = get_effort_multidim(
        cost=cost, skill=skills, diff=diffs, A=A, B=B,
    )

    high_effort = efforts > threshold
    success = prob_reward(efforts, diffs, skills, A, B) > np.random.random(n_samples)

    return skills, diffs, success, high_effort


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Function that takes in free parameters and spits out model predictions

In [5]:
def run_model(cost, A, B, u_skill,tao_skill, skill, scale,n_samples=1000):
    REWARD=1
    skills, diffs, success, high_effort= create_priors(
        n_samples =n_samples, tao_skill=tao_skill, u_skill=u_skill, cost=cost, A=A, B=B
    )

    effort_high_success = your_eff_know_skill(n_samples, cost, diffs[success & high_effort],skill, A,B) 
    effort_low_success = your_eff_know_skill(n_samples,  cost, diffs[success & (~high_effort)],skill,A,B) 
    effort_high_fail = your_eff_know_skill(n_samples,  cost, diffs[(~success) & high_effort],skill,A,B) 
    effort_low_fail = your_eff_know_skill(n_samples,  cost, diffs[(~success) & (~high_effort)],skill,A,B) 
    effort_baseline = your_eff_know_skill(n_samples,  cost, np.random.beta(.5, .5, size=n_samples),skill,A,B)

    index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"]
    d = {
        'Model' : pd.Series(
            [ 
                np.mean(effort_high_success) * scale, 
                np.mean(effort_low_success) * scale, 
                np.mean(effort_high_fail) * scale, 
                np.mean(effort_low_fail) * scale, 
                np.mean(effort_baseline)*scale
            ], index=index),
         'Human' : pd.Series(
            [107, 57, 30, 35,86], 
            index=index)}
    
    df = pd.DataFrame(d)
    rmse_val = rmse( df['Model'],df['Human'])
    return rmse_val

In [6]:
def run_model_opt(cost, A, B, u_skill,tao_skill, skill, scale,n_samples=1000):
    REWARD=1

    skills, diffs, success, high_effort= create_priors_opt(
        n_samples =n_samples, tao_skill=tao_skill, u_skill=u_skill, cost=cost, A=A, B=B
    )

    effort_high_success = your_eff_know_skill_opt(n_samples, cost, diffs[success & high_effort],skill, A,B) 
    effort_low_success = your_eff_know_skill_opt(n_samples,  cost, diffs[success & (~high_effort)],skill,A,B) 
    effort_high_fail = your_eff_know_skill_opt(n_samples,  cost, diffs[(~success) & high_effort],skill,A,B) 
    effort_low_fail = your_eff_know_skill_opt(n_samples,  cost, diffs[(~success) & (~high_effort)],skill,A,B) 
    effort_baseline = your_eff_know_skill_opt(n_samples,  cost, np.random.beta(.5, .5, size=n_samples),skill,A,B)

    model_means = np.stack(
        (effort_high_success, effort_low_success, effort_high_fail, 
         effort_low_fail, effort_baseline),
    ).mean(-1)
    model_means *= scale
    
    rmse_val = rmse(model_means, human_means)
    return rmse_val

In [7]:
def wrapped_run_model(x):
    return run_model(*x)


def wrapped_run_model_opt(x):
    return run_model_opt(*x)


inputs = np.array([0.5, 60, 6, .625, 20, .4, 200])

In [8]:
%timeit wrapped_run_model(inputs)

239 ms ± 4.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit wrapped_run_model_opt(inputs)

9.31 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Optminize function

In [10]:
key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
rmse_val= run_model_opt(**key_words)

In [11]:
inputs = np.array([0.5, 60, 6, .625, 20, .4, 200])
minimum = optimize.fmin(wrapped_run_model_opt, inputs, maxiter=1000)

In [12]:
minimum

array([   0.50191267,   60.18720961,    6.19788864,    0.6098889 ,
         20.19751562,    0.40355583,  201.59253897])

### Testing

In [13]:
cost = 0.5
B=6
A=60 
u_skill = .625 #mean of beta
tao_skill = 20 #variance of beta
skill=.4
scale = 200
threshold = .5
skills, diffs, success, high_effort=create_priors(1000, tao_skill, u_skill, cost, A, B)

In [14]:
efforthigh_success = your_eff_know_skill(1000,cost, diffs[success & high_effort],skill, A,B) 
effortlow_success = your_eff_know_skill(1000,cost, diffs[success & (~high_effort)],skill,A,B) 
efforthigh_fail = your_eff_know_skill(1000,cost, diffs[(~success) & high_effort],skill,A,B) 
effortlow_fail = your_eff_know_skill(1000,cost, diffs[(~success) & (~high_effort)],skill,A,B) 
effortbaseline = your_eff_know_skill(1000,cost, np.random.beta(.5, .5, size=100),skill,A,B)

In [15]:
scale = 200
d = {'Model' : pd.Series([np.mean(efforthigh_success)*scale, np.mean(effortlow_success)*scale, np.mean(efforthigh_fail)*scale, np.mean(effortlow_fail)*scale, np.mean(effortbaseline)*scale], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"]),
     'Human' : pd.Series([107, 57, 30, 35,85], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"])}

df = pd.DataFrame(d)
df

,Human,Model
High Effort Success,107,92.715152
Low Effort Success,57,52.620202
High Effort Failure,30,21.703030
Low Effort Failure,35,6.086869
Baseline,85,59.486869


### Testing optimized version

In [16]:
skills, diffs, success, high_effort = create_priors_opt(
    n_samples=1000, tao_skill=tao_skill, u_skill=u_skill, cost=cost, A=A, B=B,
)

default_keywords = dict(
    n_samples=1000, 
    cost=cost,
    skill=skill, 
    A=A, 
    B=B,
)

efforthigh_success = your_eff_know_skill_opt(
    diff=diffs[success & high_effort], **default_keywords,
)
effortlow_success = your_eff_know_skill_opt(
    diff=diffs[success & (~high_effort)], **default_keywords
)
efforthigh_fail = your_eff_know_skill_opt(
    diff=diffs[(~success) & high_effort], **default_keywords
)
effortlow_fail = your_eff_know_skill_opt(
    diff=diffs[(~success) & (~high_effort)], **default_keywords
)
effortbaseline = your_eff_know_skill_opt(
    diff=np.random.beta(.5, .5, size=100), **default_keywords,
)

scale = 200

model_means = np.stack(
    (efforthigh_success, effortlow_success, efforthigh_fail, 
     effortlow_fail, effortbaseline),
).mean(-1)
model_means *= scale

d = {
    'Model' : pd.Series(model_means, index=_results_index),
    'Human' : human_means
}

df = pd.DataFrame(d)
df

,Human,Model
High Effort Success,107,88.636364
Low Effort Success,57,53.577778
High Effort Failure,30,38.175758
Low Effort Failure,35,0.565657
Baseline,85,50.280808
